In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow.dataset as ds
import npc_lims

import utils

dfs = utils.get_dfs()
units = ds.dataset(npc_lims.get_cache_path('units')).filter(ds.field('default_qc') == True)
epochs = utils.add_is_task_first_column(
    utils.add_epoch_name_column(dfs['epochs'])
)

In [3]:
%time
units = pl.scan_pyarrow_dataset(units)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs


In [7]:
units = utils.add_session_id_column(units)
units.head(1).collect(streaming=True)

num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,amplitude_cutoff,drift_ptp,drift_std,drift_mad,isolation_distance,l_ratio,d_prime,peak_to_valley,peak_trough_ratio,half_width,repolarization_slope,recovery_slope,electrode_group_name,peak_channel,cluster_id,default_qc,amplitude,channels,unit_id,group_name,ccf_ap,ccf_dv,ccf_ml,structure,location,peak_electrode,spike_times,obs_intervals,electrodes,waveform_mean,waveform_sd,device_name,session_idx,date,subject_id,id,session_id
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,bool,f64,list[i64],str,str,f64,f64,f64,str,str,i64,list[f64],list[list[f64]],list[i64],list[list[f32]],list[list[f32]],str,i64,date,i64,i64,str
28697.0,7.602027,0.951613,13.738011,0.003056,2.0,0.004594,2.0,0.005,0.00005,15.307829,3.058937,4.272146,101.515747,0.743907,4.411215,0.00074,-0.260198,0.000217,749399.217529,-81810.487934,"""probeF""",127,434,true,190.644531,"[119, 120, … 136]","""620263_2022-07…","""probeF""",4700.0,2100.0,5300.0,"""ACAd""","""ACAd5""",895,"[186.06981, 231.10156, … 3815.526702]","[[40.681284, 3815.585968]]","[887, 888, … 904]","[[-0.73476, 0.09828, … -0.739439], [-0.2106, 0.72072, … -0.29952], … [0.702, -0.996841, … 0.3276]]","[[18.319595, 24.935675, … 22.614162], [18.689837, 23.324684, … 21.559978], … [18.158464, 29.49519, … 22.509325]]","""18005121302""",0,2022-07-26,620263,851,"""620263_2022-07…"


In [ ]:
units.partition_by('subject_id', 'date', 'electrode_group_name', as_dict=True)